In [1]:
import numpy as np
import pandas as pd
import joblib

from importlib import reload
from glob import glob
import os
import sys
import shutil
from sklearn.linear_model import LassoCV
import model_pedestal as pedestal

sys.path.append("..")
import kiwi_operators as kwo
import config
import data_environment as env

reload(kwo)
reload(config)
reload(env)


<module 'data_environment' from '../data_environment.py'>

In [2]:
class ModelLasso(object):
    def __init__(self):
        self.model = LassoCV(fit_intercept=True)

    def train(self, train_date_beg: int, train_date_end: int):
        train_data = pedestal.prep_feat_target(train_date_beg, train_date_end)
        train_data = train_data.dropna()

        feat_cols = list(train_data.columns)
        feat_cols.remove("time")
        feat_cols.remove("investment")
        feat_cols.remove("target")
        x_train = np.array(train_data[feat_cols])
        y_train = np.array(train_data["target"])

        self.model.fit(x_train, y_train)

    def save_model(self, file_path: str = f"{config.dir_model}/lasso.pkl"):
        joblib.dump(self.model, file_path)

    def get_model(self):
        return self.model

    def predict(self, x_data: np.array):
        return self.model.predict(x_data)



In [3]:
train_date_beg = 20160101
train_date_end = 20180101

model_lasso = ModelLasso()
model_lasso.train(train_date_beg, train_date_end)



In [16]:
valid_date_beg = 20180101
valid_date_end = 20190101

valid_data = pedestal.prep_feat_target(valid_date_beg, valid_date_end)
feat_cols = list(valid_data.columns)
feat_cols.remove("time")
feat_cols.remove("investment")
feat_cols.remove("target")
x_valid = valid_data[feat_cols].fillna(method="ffill")
x_valid = np.array(x_valid)
p_valid = model_lasso.predict(x_valid)
y_valid = np.array(valid_data["target"].fillna(method="ffill"))


In [17]:
print(np.corrcoef(p_valid, y_valid))



[[1.         0.03119736]
 [0.03119736 1.        ]]


In [11]:
print(p_valid.shape)
print(y_valid.shape)


(24985,)
(24985,)
